<a href="https://colab.research.google.com/github/ommusuc/no-4-ai-enjineering/blob/main/Homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 課題の概要

この宿題では、講義で学んだRAG(Retrieval-Augmented Generation)技術を用いて、LLMの生成
内容を改善する実践的な取り組みを行います。演習で利用したコードをベースに、独自の質問と参照文書を用
いて実験を行い、RAGの効果を定量的・定性的に評価します。
この宿題を通じて、「テストデータの作成」と「改善のプロセス」について理解を深め、実際のアプリケーション開発
に役立てることを目指します。



## 内容

宿題の内容
1. 独自の質問と参照資料の作成
- 自分で5つ以上の質問文を考案してください
- 各質問に対する回答を含む参照文書を用意してください
- 少なくとも1つは、LLMが単体では正確に答えられないような知識を含む質問にしてください
2. 実験の実施
- 演習で使用したコードをベースに、以下の2つの方法で回答を生成してください
  - ベースのLLM(RAGなし)での回答生成
  -  RAGを組み合わせた回答生成
- 回答の評価では、単純なYes/No判定でも良いです
  - より詳細な評価指標も検討していただけるとなお良いです

3. 結果分析と考察

- 生成した結果をまとめ、RAGありとRAGなしの差異を分析してください
- RAGによって回答が改善したケースと悪化したケースの両方について考察してください- 結果に基づいて、RAGの有効性と限界についての考察を記述してください

## 扱う条件

**<使用するモデル>**

2024年4月リリースのMeta-Llama-3-8B-Instructを使用する


**<参考文献>**

・ディズニー公式

・Wikipedia

・[映画『インサイド・ヘッド』あらすじ&ネタバレ！キャラクター、声優キャスト、トリビアも♪
](https://castel.jp/p/4118)


・[アナと雪の女王のあらすじをネタバレ解説!最後の結末と感動の感想まで!](https://up-tsukuba.com/frozen-summary/)

・[【作品、歴史から見る】ディズニーとピクサーの違いとは](https://liclog.net/disney-pixar-difference/)


・[映画「インサイド・ヘッド２」ネタバレあらすじ結末と感想](https://eiga-watch.com/inside-out-2/)


## 質問概要


・ディズニーに関する質問


1. アナと雪の女王」に登場する、魔法で作られた雪だるまの名前はなに？

  解答：オラフ

2. 「アナと雪の女王」でエピソードが展開される国の名前は何ですか？

  解答：アレンデール



3. ディズニー映画「インサイドヘッド」に関して教えて

  期待する解答：

  主人公少女ライリーの頭の中にいるヨロコビ、カナシミ、イカリ、ムカムカ、ビビリ5つの感情が繰り広げる冒険と1人の少女の成長を描いたストーリー

4. ディズニー映画インサイドヘッドで現在までに登場している感情のキャラクターを教えて

  解答：
  
  ヨロコビ、カナシミ、イカリ、ムカムカ、ビビリ

  シンパイ、イイナ、ハズカシ、ダリィ、ナツカシ

5. ディズニーとピクサーの違いを教えて

  解答の評価ポイント：

  設立年

  共通する点

  例)

  親会社が、ウォルトディズニーカンパニーであることなど

  異なる点

  例)

  名前・コンセプトなど

  この二つが明確にわけられているかどうかを判断基準として定性的に評価する}

In [ ]:
!pip install --upgrade transformers
!pip install google-colab-selenium
!pip install bitsandbytes

In [3]:
# HuggingFace Login
from huggingface_hub import notebook_login

notebook_login()

In [4]:
# CUDAが利用可能ならGPUを、それ以外ならCPUをデバイスとして設定
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# モデル(Llama3)の読み込み

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map="auto",
            quantization_config=bnb_config,
            torch_dtype=torch.bfloat16,
        )

In [6]:
def generate_output(query, system_prompt=None):
  if system_prompt is None:
    messages = [
        {"role": "user", "content": query},
    ]
  else:
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": query},
    ]
  input_ids = tokenizer.apply_chat_template(
      messages,
      add_generation_prompt=True,
      return_tensors="pt"
  ).to(model.device)

  terminators = [
      tokenizer.eos_token_id,
      tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = model.generate(
      input_ids,
      max_new_tokens=400,
      eos_token_id=terminators,
      do_sample=False,
      # temperature=0.6, # If do_sample=True
      # top_p=0.9,  # If do_sample=True
  )

  response = outputs[0][input_ids.shape[-1]:]
  return tokenizer.decode(response, skip_special_tokens=True)

## 1～5に関して質問を行う


### 1 アナと雪の女王」に登場する、魔法で作られた雪だるまの名前はなに？

In [ ]:
system_prompt = "質問に回答してください。必ず「日本語で回答」すること。"
question =  "アナと雪の女王」に登場する、魔法で作られた雪だるまの名前はなに？"
response = generate_output(question, system_prompt)

In [8]:
print(response)

「アナと雪の女王」シリーズの魔法で作られた雪だるまの名前は「サマンサ」。


### 2. 「アナと雪の女王」でエピソードが展開される国の名前は何ですか？

In [9]:
system_prompt = "質問に回答してください。必ず「日本語で回答」すること。"
question =  "「アナと雪の女王」でエピソードが展開される国の名前は何ですか？"
response = generate_output(question, system_prompt)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


In [10]:
print(response)

「アナと雪の女王」シリーズでは、エピソードが展開される国の名前は「アレンデール」です。


### 3.ディズニー映画「インサイドヘッド」に関して教えて

In [11]:
system_prompt = "質問に回答してください。必ず「日本語で回答」すること。"
question =  "ディズニー映画「インサイドヘッド」に関して教えて"
response = generate_output(question, system_prompt)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


In [12]:
print(response)

「インサイドヘッド」は、2015年に公開されたアメリカン・アニメーション映画「Inside Out」です。監督はピーター・ドクター、製作はピクサー・アニメーション・スタジオ。

この映画は、11歳の少女メイの脳内で起きる想像上の世界を描きます。メイの脳内には5つのエモーション（感情）が存在し、それぞれが彼女の生活に影響を与えています。5つのエモーションは、ジョー（喜び）、バリー（怒り）、ピーター（悲しみ）、サリー（哀しみ）、リグレット（後悔）です。

この映画は、子供の心の世界を描き、子供たちが経験する様々な感情を描きました。同時に、映画は、子供たちが成長する過程で必要な感情のバランスを描きました。

「インサイドヘッド」は、2015年のアカデミー賞で長編アニメーション映画賞を受賞し、世界中で大きな人気を博しました。


### 4.ディズニー映画「インサイドヘッド」で現在までに登場している感情のキャラクターを教えて

In [76]:
system_prompt = "質問に回答してください。必ず「日本語で回答」すること。"
question =  "ディズニー映画ディズニー映画インサイドヘッドで現在までに登場している感情のキャラクターを教えて"
response = generate_output(question, system_prompt)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


In [77]:
print(response)

「Inside Out」は2015年に公開されたピクサー製作のアニメーション映画です。映画の主人公は11歳の少女、リリーの脳内に存在する5つの感情のキャラクターです。

以下は、映画「Inside Out」に登場する感情のキャラクターの名前と性格です。

1. **Joy**（ハッピー）：リリーの脳内で最も強い感情。彼女は常にリリーを笑顔で見つめ、彼女の幸福を目指します。
2. **Sadness**（哀しみ）：リリーの脳内で2番目の強い感情。彼女はリリーの悲しみを理解し、彼女を慰めるために努力します。
3. **Anger**（怒り）：リリーの脳内で3番目の強い感情。彼はリリーの怒りを燃やし、彼女を守るために戦うします。
4. **Fear**（恐怖）：リリーの脳内で4番目の強い感情。彼女はリリーの恐怖を理解し、彼女を守るために警戒します。
5. **Disgust**（嫌悪）：リリーの脳内で5番目の強い感情。彼女はリリーの嫌悪を理解し、彼女を守るために警戒します。

これらのキャラクターは、リリーの脳内で活動し、彼女の感情を支配しています。映画「Inside Out」は、子供の感情の世界を描き、子供たちに自分の感情を理解することを教えることを目指しています。


### 5 ディズニーとピクサーの違いを教えて

In [15]:
def generate_output2(query, system_prompt=None):
  if system_prompt is None:
    messages = [
        {"role": "user", "content": query},
    ]
  else:
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": query},
    ]
  input_ids = tokenizer.apply_chat_template(
      messages,
      add_generation_prompt=True,
      return_tensors="pt"
  ).to(model.device)

  terminators = [
      tokenizer.eos_token_id,
      tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = model.generate(
      input_ids,
      max_new_tokens=2000,
      eos_token_id=terminators,
      do_sample=False,
      # temperature=0.6, # If do_sample=True
      # top_p=0.9,  # If do_sample=True
  )

  response = outputs[0][input_ids.shape[-1]:]
  return tokenizer.decode(response, skip_special_tokens=True)

In [16]:
system_prompt = "質問に回答してください。必ず「日本語で回答」すること。"
question =  "ディズニーとピクサーの違いを教えて"
response = generate_output2(question, system_prompt)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


In [17]:
response

'ディズニーとピクサーは、両方ともアニメーション製作会社ですが、歴史的には独立した存在で、現在でも異なる組織体制を持っています。\n\nディズニーは、ウォルト・ディズニーが創業したアニメーション製作会社で、1930年代に設立されました。ディズニーは、ウォルト・ディズニー自身が創作したキャラクター「ミッキー・マウス」や「ドナルド・ダック」などの人気キャラクターを生み出し、映画「白雪姫」や「シンデレラ」などのクラシック・アニメーション映画を製作しました。\n\n一方、ピクサーは、スティーブ・ジョブズとジョン・ラタザンが1986年に設立したアニメーション製作会社です。ピクサーは、初期にはディズニーとの提携関係にあり、「トイ・ストーリー」や「モンスターズ・インク」などの映画を製作しました。2006年にディズニーがピクサーを買収し、現在はディズニーの一部門として運営されています。\n\nピクサーの特徴として、ストーリー性の強いアニメーション映画を製作し、「トイ・ストーリー」や「インクレディブルズ」などの大ヒット映画を生み出しています。また、ピクサーは、CGI（コンピューター・ジェネレーテッド・イメージ）技術を積極的に使用し、3Dアニメーション映画を製作しています。\n\n以上のように、ディズニーとピクサーは、歴史的には独立した存在で、現在でも異なる組織体制を持っています。ただし、ピクサーがディズニーに買収されたことで、両方の会社は、より強力なアニメーション製作能力を有することになりました。'

## RAG実装

### アナ雪

**初期RAG実装（ベーシックアプローチ）**:
* **ドキュメント処理**: Wikipediaにおける「アナと雪の女王」のストーリーを参照
* **分割方法**: 「。」（句点）で区切られた文単位でテキストを分割
* **検索手法**: シンプルな類似度ベースの検索でクエリに関連する文を抽出
* **制約条件**: モデルの入力トークン制限に収まるよう関連文のみを選択

In [44]:
from sentence_transformers import SentenceTransformer

emb_model = SentenceTransformer("infly/inf-retriever-v1-1.5b", trust_remote_code=True)
# In case you want to reduce the maximum length:
emb_model.max_seq_length = 8192

In [51]:
with open("/content/frozen.txt", "r") as f:
  raw_writedown = f.read()

In [52]:
# ドキュメントを用意する。
documents = [text.strip() for text in raw_writedown.split("。")]
print("ドキュメントサイズ: ", len(documents))
print("ドキュメントの例: \n", documents[42])

ドキュメントサイズ:  85
ドキュメントの例: 
 エルサは成長するにつれて力が増し、この力を世間から隠すために城の門や窓を閉め切り、エルサは誰にも会わない生活を強いられました


In [55]:
# Retrievalの実行
question = "アナと雪の女王」に登場する、雪だるまの名前はなに？"

query_embeddings = emb_model.encode([question], prompt_name="query")
document_embeddings = emb_model.encode(documents)

# 各ドキュメントの類似度スコア
scores = (query_embeddings @ document_embeddings.T) * 100
print(scores.tolist())

[[71.68010711669922, 76.77472686767578, 73.00437927246094, 69.40055084228516, 74.25717163085938, 74.01288604736328, 65.18295288085938, 72.58943176269531, 67.93061065673828, 65.6526107788086, 75.89250946044922, 63.358367919921875, 70.46659851074219, 67.56652069091797, 70.9630355834961, 69.24102020263672, 74.64913177490234, 77.0921859741211, 73.70491027832031, 74.6927490234375, 73.51960754394531, 76.43871307373047, 73.21900177001953, 72.93183135986328, 75.68218231201172, 74.4514389038086, 73.74519348144531, 74.84251403808594, 71.94361114501953, 75.30755615234375, 72.61013793945312, 75.54000854492188, 67.29603576660156, 69.77497863769531, 71.12789916992188, 67.55241394042969, 71.79352569580078, 69.26045989990234, 75.23695373535156, 72.74004364013672, 73.70716094970703, 75.26972198486328, 66.18231964111328, 72.33055877685547, 63.00920104980469, 74.35406494140625, 67.72913360595703, 65.63951110839844, 66.38382720947266, 67.02577209472656, 73.99362182617188, 74.07969665527344, 69.81304931640

In [56]:
topk = 5
for i, index in enumerate(scores.argsort()[0][::-1][:topk]):
  print(f"取得したドキュメント{i+1}: (Score: {scores[0][index]})")
  print(documents[index], "\n\n")

取得したドキュメント1: (Score: 77.98040008544922)
やむなく引き返したアナたちでしたが、クリストフがアナの髪の毛が凍り始めていることに気づき、クリストフの友人であるトロールたちのもとに連れていきます 


取得したドキュメント2: (Score: 77.0921859741211)
旅の途中、アナは氷を売る山男のクリストフとその忠実なトナカイ、スヴェンに出会います 


取得したドキュメント3: (Score: 77.08509826660156)
アナはハンス王子と思い浮かべ、クリストフはアナを連れて急いでアレンデールに戻ります 


取得したドキュメント4: (Score: 76.77472686767578)
姉のエルサは、生まれつき氷や雪を自在に操る魔法の力を持っており、妹のアナと一緒にその力を使って遊ぶ日々を送っていました 


取得したドキュメント5: (Score: 76.43871307373047)
この個性豊かな仲間たちと共に、アナはエルサのいる北山の氷の宮殿を目指します 




In [57]:
references = "\n".join(["* " + documents[i] for i in scores.argsort()[0][::-1][:topk]])
system_prompt = "質問に回答してください。必ず「日本語で回答」すること。また、与えられる資料を参考にして回答すること。"
question =  f"[参考資料]\n{references}\n\n[質問] アナと雪の女王」に登場する、雪だるまの名前はなに？"
response = generate_output(question, system_prompt)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


In [58]:
response

'参考資料に基づいて、雪だるまの名前はスヴェンです。スヴェンはクリストフの忠実なトナカイです。'

### インサイドヘッド


In [109]:
with open("/content/inside out.txt", "r") as f:
  raw_writedown = f.read()

In [110]:
# ドキュメントを用意する。
documents = [text.strip() for text in raw_writedown.split("。")]
print("ドキュメントサイズ: ", len(documents))
print("ドキュメントの例: \n", documents[42])

ドキュメントサイズ:  88
ドキュメントの例: 
 そして家族と再会し、「本当はミネソタが好き」「前の家に帰りたい」と涙ながらに自分の本心を打ち明けたライリーに喜びと悲しみが混じった思い出が生まれ、その思い出ボールが全ての感情の島を復活させる


In [111]:
# Retrievalの実行
question = "ディズニー映画「インサイドヘッド」に関して教えて"

query_embeddings = emb_model.encode([question], prompt_name="query")
document_embeddings = emb_model.encode(documents)

# 各ドキュメントの類似度スコア
scores = (query_embeddings @ document_embeddings.T) * 100
print(scores.tolist())

[[71.99604797363281, 72.82036590576172, 72.24185180664062, 72.9712142944336, 72.67969512939453, 71.38568115234375, 73.17573547363281, 70.40386199951172, 72.04653930664062, 71.52470397949219, 72.65802764892578, 70.99932098388672, 70.65118408203125, 72.61425018310547, 71.7345962524414, 74.59403228759766, 69.83406829833984, 73.96324920654297, 72.40596771240234, 70.08460998535156, 72.54803466796875, 73.76397705078125, 72.1924057006836, 69.9460678100586, 69.87785339355469, 70.47522735595703, 73.14948272705078, 72.36266326904297, 70.67862701416016, 71.02726745605469, 71.92675018310547, 71.62003326416016, 72.58513641357422, 72.20272827148438, 72.19515991210938, 71.44235229492188, 71.62521362304688, 71.39739227294922, 73.59555053710938, 73.2077865600586, 73.19425201416016, 70.6268081665039, 73.2479476928711, 71.24732971191406, 71.10594177246094, 85.54563903808594, 73.34747314453125, 71.83477020263672, 73.07112121582031, 68.52426147460938, 70.54220581054688, 71.02923583984375, 68.86894989013672

In [112]:
topk = 5
for i, index in enumerate(scores.argsort()[0][::-1][:topk]):
  print(f"取得したドキュメント{i+1}: (Score: {scores[0][index]})")
  print(documents[index], "\n\n")

取得したドキュメント1: (Score: 85.54563903808594)
前作『インサイド・ヘッド』の出来事から2年 


取得したドキュメント2: (Score: 74.59403228759766)
それを阻害しようとしたヨロコビと、引き留めようとしたカナシミが揉め合ってるうちに「特別な思い出」が散らばり、「長期記憶の保管場所」へ向かうチューブに吸い込まれる 


取得したドキュメント3: (Score: 73.96324920654297)
深い谷のような「記憶のゴミ捨て場」で司令部と隔てられ、迷路のように複雑な「長期記憶の保管場所」へ飛ばされてしまったヨロコビとカナシミは、そこをくぐり抜けて「性格の島」から司令部へ帰ろうとする 


取得したドキュメント4: (Score: 73.76397705078125)
ヨロコビとカナシミは遠回りを強いられ「友情の島」を通ろうとするが、ミネソタでの親友メグに新しい友達ができたと知ったライリーがカッとなった瞬間、「友情の島」まで崩れ落ちてしまった 


取得したドキュメント5: (Score: 73.59555053710938)
ヨロコビが絶望に暮れながら手元の特別な思い出を眺めていると、ある思い出に悲しみと喜びが混じっていることに気付く 




In [113]:
references = "\n".join(["* " + documents[i] for i in scores.argsort()[0][::-1][:topk]])
system_prompt = "質問に回答してください。必ず「日本語で回答」すること。また、与えられる資料を参考にして回答すること。"
question =  f"[参考資料]\n{references}\n\n[質問] ディズニー映画「インサイドヘッド」に関して教えて"
response = generate_output2(question, system_prompt)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


In [114]:
print(response)

日本語で回答します。

「インサイド・ヘッド」はディズニー映画の題材であり、前作の出来事から2年が経過した後、ヨロコビとカナシミが「特別な思い出」が散らばり、「長期記憶の保管場所」へ吸い込まれるというストーリーが展開されます。

その後、ヨロコビとカナシミは「記憶のゴミ捨て場」で迷路のように複雑な「長期記憶の保管場所」へ飛ばされ、「性格の島」から司令部へ帰ろうとするが、途中で「友情の島」に到達し、新しい友達ができたと知ったライリーがカッとなった瞬間、「友情の島」まで崩れ落ちてしまう。

ヨロコビは絶望に暮れながら手元の特別な思い出を眺めていると、ある思い出に悲しみと喜びが混じっていることに気付くという展開が描かれます。


まあ全体名流れはだいぶ抑えているので及第点

ただインサイドヘッド2の内容も入っており、冒頭の部分が違和感のあるものになっているのがよくない

In [115]:
# Retrievalの実行
question = "ディズニー映画インサイドヘッドで現在までに登場している感情のキャラクターを教えて"

query_embeddings = emb_model.encode([question], prompt_name="query")
document_embeddings = emb_model.encode(documents)

# 各ドキュメントの類似度スコア
scores = (query_embeddings @ document_embeddings.T) * 100
print(scores.tolist())

[[70.69475555419922, 76.75619506835938, 73.76100158691406, 78.34037780761719, 73.34410858154297, 72.26178741455078, 74.60418701171875, 70.3356704711914, 70.32308959960938, 72.22406005859375, 73.1828842163086, 69.47567749023438, 69.64305877685547, 72.95466613769531, 72.13855743408203, 73.14593505859375, 67.9430923461914, 72.97962951660156, 73.95629119873047, 70.55797576904297, 71.66218566894531, 72.4122314453125, 71.4085922241211, 70.20298767089844, 70.7712173461914, 69.10167694091797, 71.6400375366211, 70.54883575439453, 69.67788696289062, 71.14396667480469, 72.07239532470703, 69.81017303466797, 70.69886016845703, 70.13787078857422, 71.38512420654297, 69.44666290283203, 69.7607192993164, 74.6383285522461, 75.1597900390625, 73.37098693847656, 70.43350982666016, 70.12528991699219, 75.558349609375, 74.91167449951172, 71.27359008789062, 81.3674545288086, 71.59246063232422, 68.53385162353516, 76.95418548583984, 67.00163269042969, 71.32134246826172, 71.73860931396484, 68.80903625488281, 72.3

In [116]:
topk = 5
for i, index in enumerate(scores.argsort()[0][::-1][:topk]):
  print(f"取得したドキュメント{i+1}: (Score: {scores[0][index]})")
  print(documents[index], "\n\n")

取得したドキュメント1: (Score: 81.3674545288086)
前作『インサイド・ヘッド』の出来事から2年 


取得したドキュメント2: (Score: 78.34037780761719)
彼ら5つの感情はヨロコビをリーダーとして、ライリーの頭の中にある「司令部」で彼女の人生を幸せにするために日々奮闘している 


取得したドキュメント3: (Score: 77.17964935302734)
一方、ライリーの頭の司令部には新たなる感情が誕生していました 


取得したドキュメント4: (Score: 77.08409118652344)
“不安”を司るシンパイ、“嫉妬”を司るイイナー、“羞恥心”を司るハズカシ、“倦怠感”を司るダリィです 


取得したドキュメント5: (Score: 76.95418548583984)
ライリーの頭の中の“司令部”では彼女の人格形成を司る5つの感情、前向きでポジティブなヨロコビ、どこまでもネガティブなカナシミ、短気で怒りっぽいイカリ、嫌なことを遠ざけるのが役割のムカムカ、危険や恐怖から身を守ることが役割のビビリがライリーに指示を出しており、ホッケーの試合に出場するライリーをアシストしていました 




In [117]:
references = "\n".join(["* " + documents[i] for i in scores.argsort()[0][::-1][:topk]])
system_prompt = "質問に回答してください。必ず「日本語で回答」すること。また、与えられる資料を参考にして回答すること。"
question =  f"[参考資料]\n{references}\n\n[質問] ディズニー映画インサイドヘッドで現在までに登場している感情のキャラクターを教えて"
response = generate_output2(question, system_prompt)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


In [118]:
print(response)

インサイド・ヘッドの前作で登場した5つの感情のキャラクターは、以下の通りです。

1. ヨロコビ (Joy) - ポジティブで前向きの感情
2. カナシミ (Sadness) - ネガティブで悲しい感情
3. イカリ (Anger) - 短気で怒りっぽい感情
4. ムカムカ (Disgust) - 嫌なことを遠ざけるのが役割の感情
5. ビビリ (Fear) - 危険や恐怖から身を守ることが役割の感情

また、新たに登場した感情のキャラクターとして、以下の5つが挙げられます。

1. シンパイ (Anxiety) - 不安を司る感情
2. イイナー (Envy) - 嫉妬を司る感情
3. ハズカシ (Shame) - 羞恥心を司る感情
4. ダリィ (Boredom) - 倦怠感を司る感情

以上、インサイド・ヘッドの感情のキャラクターを紹介しました。


### ディズニーとピクサーの違い


In [119]:
with open("/content/diff_disnet and pixer.txt", "r") as f:
  raw_writedown = f.read()

In [120]:
# ドキュメントを用意する。
documents = [text.strip() for text in raw_writedown.split("。")]
print("ドキュメントサイズ: ", len(documents))
print("ドキュメントの例: \n", documents[42])

ドキュメントサイズ:  52
ドキュメントの例: 
 ジョン・ラセターはあるインタビューでピクサー映画についてこう答えました


In [121]:
# Retrievalの実行
question = "ディズニーとピクサーの違いについて教えて"

query_embeddings = emb_model.encode([question], prompt_name="query")
document_embeddings = emb_model.encode(documents)

# 各ドキュメントの類似度スコア
scores = (query_embeddings @ document_embeddings.T) * 100
print(scores.tolist())

[[89.8675765991211, 79.74027252197266, 69.57640075683594, 77.90788269042969, 88.58875274658203, 80.5750503540039, 81.41664123535156, 75.16128540039062, 88.26056671142578, 85.68116760253906, 83.69738006591797, 85.95860290527344, 86.10472869873047, 81.15034484863281, 72.28981018066406, 79.84545135498047, 74.86311340332031, 70.91935729980469, 79.96627807617188, 78.491943359375, 81.04409790039062, 77.6712417602539, 80.1758041381836, 80.07783508300781, 70.33513641357422, 68.0876235961914, 84.94292449951172, 73.96065521240234, 81.21971130371094, 81.50310516357422, 71.49153137207031, 84.13323974609375, 83.69738006591797, 82.7682113647461, 84.28376770019531, 77.88427734375, 70.86446380615234, 76.29939270019531, 78.47893524169922, 75.48715209960938, 84.75218200683594, 75.29816436767578, 79.37081146240234, 78.3371353149414, 73.8866958618164, 81.09334564208984, 77.65509033203125, 78.30094146728516, 81.52752685546875, 80.5530776977539, 85.74539184570312, 60.29546356201172]]


In [122]:
topk = 5
for i, index in enumerate(scores.argsort()[0][::-1][:topk]):
  print(f"取得したドキュメント{i+1}: (Score: {scores[0][index]})")
  print(documents[index], "\n\n")

取得したドキュメント1: (Score: 89.8675765991211)
結論から言ってしまうとディズニーとピクサーの違いは「製作スタジオ(会社)の違い」です 


取得したドキュメント2: (Score: 88.58875274658203)
つまり、このページで扱うディズニーとピクサーの違いは正式にいうと
『ウォルト・ディズニー・アニメーション・スタジオ』と
『ピクサー・アニメーション・スタジオ』の違いという訳です 


取得したドキュメント3: (Score: 88.26056671142578)
まとめるとディズニー(ウォルト・ディズニー・アニメーション・スタジオ)とピクサーは同じ『ウォルト・ディズニー・カンパニー』を親としている子会社同士ですが、映画を製作するスタジオ(会社)としては別ということです 


取得したドキュメント4: (Score: 86.10472869873047)
ここではピクサーがディズニーの傘下に入るまでの歴史を見ていきましょう 


取得したドキュメント5: (Score: 85.95860290527344)
そんな別の会社だったディズニーとピクサーですが全くの無関係だった訳ではありません 




In [123]:
references = "\n".join(["* " + documents[i] for i in scores.argsort()[0][::-1][:topk]])
system_prompt = "質問に回答してください。必ず「日本語で回答」すること。また、与えられる資料を参考にして回答すること。"
question =  f"[参考資料]\n{references}\n\n[質問] ディズニーとピクサーの違いについて教えて"
response = generate_output(question, system_prompt)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


In [124]:
print(response)

ディズニーとピクサーの違いは、製作スタジオ(会社)の違いです。正式には、『ウォルト・ディズニー・アニメーション・スタジオ』と『ピクサー・アニメーション・スタジオ』の違いです。両者は同じ『ウォルト・ディズニー・カンパニー』を親としている子会社同士ですが、映画を製作するスタジオ(会社)としては別の存在です。


# 実装の感想


全体として、RAG実装前よりも正確な情報かつ入れたテキストファイルにもよるが正確な結果を返していることがわかりRAG手法が効果的であることが分かった

ただ、アナ雪のオラフのようにテキストファイルの中に似た属性をもつものが存在するとどうしてもより質問に対する類似度が高い方を優先してしまう傾向があることが分かったのでRAGの実装方法及び、テキストファイル内の情報の均一性などが非常に重要であると感じた

また、ただ参照してきたサイトの文書を丸ごと持ってきただけなのにもかかわらず「インサイドヘッド」のようにキャラクターの出現を明確に分けることもできるようになっておりその点に関してはLLMのすごさを感じたのでこれから実際に自分が内部でなにを行っているかなどをしっかりと追っていけるようになりたいと感じた


# これから

今回の実装で学んだことは主に下記の二つ

1. RAGの実装方法の多様さと懸念点

2. LLMへの質問を明確にする根拠


この二つを今回自分で実装することでさらに、理解が深まりました

特に、最後の方ではなかなかうまくいかない点もあったので授業資料を負いつつほかの講座もとることでなんとか解決手法をこれから探していければと感じました

RAGに関していえば、具体的にはどのような情報を読み込ませることが重要なのだろうかもしくは莫大な量のテキストファイルさえあればいいのかなど基本的なLLMの知識がないのでそこをまず学習しようと感じました

また、自分自身がLLMを実装する機会もしくはさらに先のAIモデルを実装することがあるかもしれないのでその起点になった気がしました

